In [1]:
import sys
import os
import numpy as np

from codingTools import *


In [2]:
######LOAD RAW IMAGE######
#dtype --> > stands for big endian, i stands for integer, 2 stands for 2 bytes
originalFile="../images/03508649.1_512_512_2_0_12_0_0_0d.raw" #int16
#originalFile="../images/03508649.1_128_128_1_0_8_0_0_0.raw"
#originalFile="../images/n1_GRAY.1_2560_2048_1_0_8_0_0_0.raw" #uint8
ImageRaw = load_image_raw(originalFile, 1, 512, 512, ">i2") #https://numpy.org/doc/stable/reference/arrays.dtypes.html
#ImageRaw = load_image_raw(originalFile, 1, 128, 128, "uint8")
#ImageRaw = load_image_raw(originalFile, 1, 2560, 2048, "uint8") #https://numpy.org/doc/stable/reference/arrays.dtypes.html
original_image = ImageRaw.load_image_raw()
##############################

In [3]:
########STATISTICS##########
statistics_data = statistics(original_image)
entropy = statistics_data.compute_entropy()
print("order zero entropy of the image: ",entropy)

order zero entropy of the image:  10.431490855522489


In [4]:
print(original_image)

[[[356 359 386 ... 415 398 389]
  [351 362 363 ... 417 405 398]
  [355 363 394 ... 419 398 414]
  ...
  [361 375 414 ... 436 417 414]
  [331 341 374 ... 442 380 406]
  [332 344 354 ... 406 398 368]]]


In [5]:
######SAVE RAW IMAGE######
# decodedFile="../images/n1_GRAY.1_2560_2048_1_0_8_0_0_0_copy.raw"
# ImageRawDecoded = save_image_raw(decodedFile)
# ImageRawDecoded.save_image_raw(original_image,"uint8")
###########

In [6]:
### Quantize ###
quantize = Quantizer(1, 10)
image_data_quantized = ImageRaw.get_image_structure_empty()
quantize.quantize(original_image, image_data_quantized)
quant_statics = statistics(image_data_quantized).compute_entropy()
print(f"Entropia: {quant_statics}")


Check Quantize: [[[35. 35. 38. ... 41. 39. 38.]
  [35. 36. 36. ... 41. 40. 39.]
  [35. 36. 39. ... 41. 39. 41.]
  ...
  [36. 37. 41. ... 43. 41. 41.]
  [33. 34. 37. ... 44. 38. 40.]
  [33. 34. 35. ... 40. 39. 36.]]]
Entropia: 7.1184477268956465


In [7]:
### Dequantize ###
dequantize = Quantizer(1, 10)
image_data_dequantized = ImageRaw.get_image_structure_empty()
dequantize.dequantize(image_data_quantized, image_data_dequantized)

In [8]:
### PAE ###
pae = Metrics(original_image, image_data_dequantized)
print(f"Pae: {pae.calculate_pae()}")
print(pae.PAE_data)


Pae: 9.0
9.0


In [9]:
### MSE ###
mse = Metrics(original_image, image_data_dequantized)
print(f"Mse: {mse.calculate_mse()}")

Mse: 28.504417419433594


In [10]:
### PSNR ###
psnr = Metrics(original_image, image_data_dequantized)
print(f"Psnr: {psnr.calculate_psnr()} dB")

Psnr: 57.6980772672628 dB


In [11]:
### S-transform forward ###
array = [1,2,3,4,8,6,10,8]
transform = Wavelet(array, 1)
array_transform = transform.s_tranform_forward(array)
print(f"Array modificado: {array_transform}")
    

Array modificado: [ 1  3  7  9 -1 -1  2  2]


In [12]:
### S-transform Inverse ###
array_inv = transform.s_transform_inverse(array_transform)
print(f"Array invertido: {array_inv}")


Array invertido: [ 1  2  3  4  8  6 10  8]


In [13]:
### S-transform Forward image ###
transform_image = Wavelet(original_image, 3)
#print(len(original_image[0][0]))
transform_forward = transform_image.handle_transform_forward()
print(transform_forward)


[[436 525 615 ...  51   9  -2]
 [553 666 783 ...  -6   5  11]
 [641 783 920 ...  22  -8 -14]
 ...
 [ 11  25  -1 ...  -1 -53  31]
 [ 18  15  11 ...  69 -17  -8]
 [  8   2 -12 ...  14  25 -56]]


In [14]:
### S-transform Reverse image ###
transform_image_reverse = Wavelet(transform_forward, 3)
transform_reverse = transform_image_reverse.handle_transform_inverse()

In [15]:
print(transform_reverse)

[[[356 359 386 ... 415 398 389]
  [351 362 363 ... 417 405 398]
  [355 363 394 ... 419 398 414]
  ...
  [361 375 414 ... 436 417 414]
  [331 341 374 ... 442 380 406]
  [332 344 354 ... 406 398 368]]]


In [16]:
print(original_image)
 

[[[356 359 386 ... 415 398 389]
  [351 362 363 ... 417 405 398]
  [355 363 394 ... 419 398 414]
  ...
  [361 375 414 ... 436 417 414]
  [331 341 374 ... 442 380 406]
  [332 344 354 ... 406 398 368]]]


### Classe 17/09
Predicció:
X = 1,2,3,4,8,6,10,8
Causal
Pi = {0 si i=0, X(i-1) si i>0}
X'i = Xi - Pi
P = 0,1,2,3,4,8,6,10 (Predicció)
X' = 1,1,1,1,4,-2,4,-2
#
###### TODO
Predict sobre una matriu. -> ha de ser invertible -> img de 512 512
Calcular entropia del resultat X' -> H(x') <= H(x)

In [23]:
pred = Prediction(original_image)
mat_pred = pred.predict_image()
statistics_data_pred = statistics(mat_pred)
entropy_pred = statistics_data_pred.compute_entropy()
print("Original entropy ",entropy)
print("Pred entropy: ", entropy_pred)
#invert = Prediction(mat_pred)
image_rebuilt = invert_image(mat_pred)
print(f"\nLa imagen devuelta es:\n{image_rebuilt}")
print(f"\nLa imagen original es:\n{original_image}")


Original entropy  10.431490855522489
Pred entropy:  7.017758012788629

La imagen devuelta es:
[[[356 359 386 ... 415 398 389]
  [351 362 363 ... 417 405 398]
  [355 363 394 ... 419 398 414]
  ...
  [361 375 414 ... 436 417 414]
  [331 341 374 ... 442 380 406]
  [332 344 354 ... 406 398 368]]]

La imagen original es:
[[[356 359 386 ... 415 398 389]
  [351 362 363 ... 417 405 398]
  [355 363 394 ... 419 398 414]
  ...
  [361 375 414 ... 436 417 414]
  [331 341 374 ... 442 380 406]
  [332 344 354 ... 406 398 368]]]
